# ClipCard Simulation: Clinical/SBAR
**Purpose:** This notebook runs a Monte Carlo simulation to estimate the effectiveness of ClipCard parameters for a clinical scenario, such as monitoring a patient on a new anticoagulation protocol.
It simulates a patient's lab results (INR) over time and checks if they would trigger the `kill_criteria` defined in a ClipCard. This helps quantify the risk of adverse events and the effectiveness of the safety backstops.

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport json# Load example ClipCardtry:    with open('../../examples/clinical-sbar-test.clipcard.json', 'r') as f:        example_card = json.load(f)    print(f"Loaded example ClipCard: {example_card['id']}")except FileNotFoundError:    print("No example ClipCard found. Using synthetic data.")    example_card = {        "id": "SIM-CLINICAL-001",        "kill_criteria": [            {"condition": "INR > 3.5", "action": "Stop dose; notify Attending"}        ]    }

### Simulation Parameters
Configure simulation runs for patient INR monitoring.

In [ ]:
# Simulation configurationN_SIMULATIONS = 500  # Number of simulated patientsN_TIMESTEPS = 48     # Number of hours to simulateSTARTING_INR = 2.0   # Therapeutic INR targetINR_VOLATILITY = 0.1 # Standard deviation of INR change per hourKILL_THRESHOLD = 3.5 # From the ClipCardprint(f"Running {N_SIMULATIONS} simulations for {N_TIMESTEPS} hours each...")print(f"Starting INR: {STARTING_INR}")print(f"INR Volatility (SD per hour): {INR_VOLATILITY}")print(f"Kill Threshold: > {KILL_THRESHOLD}")

### Monte Carlo Simulation
Run simulations of patient INR using a random walk model.

In [ ]:
results = []all_paths = []for i in range(N_SIMULATIONS):    inr_path = [STARTING_INR]    triggered = False    for t in range(1, N_TIMESTEPS):        # Random walk model for INR        next_inr = inr_path[-1] + np.random.normal(0, INR_VOLATILITY)        if next_inr < 0.5: next_inr = 0.5 # Clamp at a reasonable minimum        inr_path.append(next_inr)        if next_inr > KILL_THRESHOLD:            triggered = True            # In a real simulation, the path would stop here, but we continue to see the full path        all_paths.append(inr_path)    results.append({        'sim_id': i,        'max_inr': max(inr_path),        'triggered': triggered    })df = pd.DataFrame(results)print(f"
Simulation complete!")print(f"Patients who triggered the kill criterion: {df['triggered'].sum()} ({df['triggered'].mean()*100:.1f}%)")

### Visualization
Plot simulated INR paths and the distribution of maximum INR values.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))# Plot a few sample INR pathsfor i in range(min(20, N_SIMULATIONS)):    color = 'red' if max(all_paths[i]) > KILL_THRESHOLD else 'grey'    alpha = 0.9 if color == 'red' else 0.3    axes[0].plot(all_paths[i], color=color, alpha=alpha)axes[0].axhline(KILL_THRESHOLD, color='red', linestyle='--', label=f'Kill threshold (> {KILL_THRESHOLD})')axes[0].set_xlabel('Time (hours)')axes[0].set_ylabel('INR')axes[0].set_title('Sample Simulated Patient INR Paths')axes[0].legend()axes[0].set_ylim(bottom=0)# Plot the distribution of maximum INR valuesaxes[1].hist(df['max_inr'], bins=30, alpha=0.7, edgecolor='black')axes[1].axvline(KILL_THRESHOLD, color='red', linestyle='--', label=f'Kill threshold (> {KILL_THRESHOLD})')axes[1].set_xlabel('Maximum INR reached during simulation')axes[1].set_ylabel('Frequency (Number of Patients)')axes[1].set_title('Distribution of Maximum INR')axes[1].legend()plt.tight_layout()plt.show()print(f"
** Interpretation **")print(f"In {N_SIMULATIONS} simulated patients over {N_TIMESTEPS} hours:")print(f"- The INR kill criterion was triggered for {df['triggered'].mean()*100:.1f}% of patients.")print(f"- This suggests that with an INR volatility of {INR_VOLATILITY}, there is a significant chance of the INR exceeding the safety threshold.")